In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Leer el archivo Excel
ruta_archivo = r"C:\Users\USUARIO\Downloads\extorsion_11.xlsx"
df = pd.read_excel(ruta_archivo)

In [3]:
# Eliminar las primeras 9 filas que contienen información no relevante
df = df.drop(index=range(8))

In [4]:
# Resetear el índice para que comience desde 0
df.reset_index(drop=True, inplace=True)

In [5]:
# Asignar los nombres correctos a las columnas
df.columns = df.iloc[0]  # Asigna la primera fila como nombres de columnas
df = df.drop(index=0)  # Elimina la fila que ahora es redundante

In [6]:
# Resetear el índice nuevamente
df.reset_index(drop=True, inplace=True)

In [7]:
# Eliminar filas adicionales que puedan estar en blanco o no sean relevantes
df = df.dropna(how='all')

In [8]:
# Asegúrate de que las columnas clave están correctamente nombradas
print(df.columns)

Index(['ARMA MEDIO ', 'DEPARTAMENTO ', 'MUNICIPIO ', 'FECHA HECHO ', 'GENERO ',
       '*AGRUPA_EDAD_PERSONA', 'CODIGO DANE ', 'CANTIDAD '],
      dtype='object', name=0)


In [9]:
# Mostrar los primeros registros para verificar que todo esté bien
print(df.head())

0      ARMA MEDIO  DEPARTAMENTO  MUNICIPIO          FECHA HECHO   \
0  CARTA EXTORSIVA     ANTIOQUIA      Anorí  2023-01-08 00:00:00   
1  CARTA EXTORSIVA     ANTIOQUIA      Anorí  2023-05-30 00:00:00   
2  CARTA EXTORSIVA     ANTIOQUIA      Bello  2023-01-07 00:00:00   
3  CARTA EXTORSIVA     ANTIOQUIA      Bello  2023-02-06 00:00:00   
4  CARTA EXTORSIVA     ANTIOQUIA      Bello  2023-02-23 00:00:00   

0       GENERO  *AGRUPA_EDAD_PERSONA CODIGO DANE  CANTIDAD   
0     MASCULINO              ADULTOS     05040000         1  
1  NO REPORTADO         NO REPORTADO     05040000         1  
2      FEMENINO              ADULTOS     05088000         1  
3     MASCULINO              ADULTOS     05088000         1  
4     MASCULINO              ADULTOS     05088000         1  


In [19]:
# Asignar manualmente los nombres correctos a las columnas
df.columns = ['ARMA MEDIO', 'DEPARTAMENTO', 'MUNICIPIO', 'FECHA HECHO', 'GENERO', 
              'AGRUPA_EDAD_PERSONA', 'CODIGO DANE', 'CANTIDAD']

In [21]:
# Agrupar por Departamento, Municipio, y contar el número de delitos
grupo_departamento = df.groupby(['DEPARTAMENTO']).agg(
    numero_de_casos_de_extorsion=('CANTIDAD', 'sum')
).reset_index()

In [23]:
# Mostrar los primeros resultados
print(grupo_departamento.head())

  DEPARTAMENTO numero_de_casos_de_extorsion
0     AMAZONAS                            1
1    ANTIOQUIA                         1918
2       ARAUCA                           75
3    ATLÁNTICO                         1362
4      BOLÍVAR                          267


In [25]:
grupo_departamento.to_excel('C:/Users/USUARIO/OneDrive/Escritorio/Datos Policiales/Prueba1/resultado_numero_de_casos_de_extorsion.xlsx', index=False)

In [ ]:
# Calcular el total de homicidios por Departamento
total_homicidios_depto = grupo_departamento.groupby('DEPARTAMENTO').agg(
    total_de_delitos_sexuales=('numero_de_delitos_sexuales', 'sum')
).reset_index()

In [ ]:
# Unir el total de homicidios por Departamento con el DataFrame original
df_final = pd.merge(grupo_departamento, total_homicidios_depto, on='DEPARTAMENTO')

In [ ]:
# Ordenar los datos por Departamento y Municipio
df_final = df_final.sort_values(by=['DEPARTAMENTO', 'MUNICIPIO',]).reset_index(drop=True)

In [ ]:
for depto in df_final['DEPARTAMENTO'].unique():
    print(f"\nDepartamento: {depto}")
    municipios = df_final[df_final['DEPARTAMENTO'] == depto]['MUNICIPIO'].unique()
    for municipio in municipios:
        total = df_final[(df_final['DEPARTAMENTO'] == depto) & (df_final['MUNICIPIO'] == municipio)]
        for _, row in total.iterrows():
            print(f"  Municipio: {municipio}", f" -  Delitos sexuales: {row['numero_de_delitos_sexuales']}")
    total_de_delitos_sexuales = df_final[df_final['DEPARTAMENTO'] == depto]['total_de_delitos_sexuales'].iloc[0]
    print(f"  Total de delitos sexuales en {depto}: {total_de_delitos_sexuales}")

In [ ]:
import seaborn as sns
# Graficar los totales por departamento
plt.figure(figsize=(10, 6))
plt.barh(total_homicidios_depto['DEPARTAMENTO'], total_homicidios_depto['total_de_delitos_sexuales'])
plt.xlabel('Número de delitos sexuales')
plt.title('Grafica de delitos sexuales por Departamento')
plt.show()

In [ ]:
df_final.to_excel('C:/Users/USUARIO/OneDrive/Escritorio/Datos Policiales/Prueba1/resultado_delitos_sexuales22.xlsx', index=False)